### Generates the Records_df and from that computes and exports Frequency_df
### Frequency_df is the only one directly used by the Pathapp

In [1]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from sample_data import type_list,value_sign,day,adequacy_list,choices,last_week_date, last_month_date,results_list, genotype_list

## Generates the numer of tests processed in a given date

In [3]:
### this function defines how busy each day of the week is
def busy_days(n:int):
    ans=100
    if n==6:
        ans=0
    return ans
    
    
    

In [4]:
### this function defines how busy each month is
def busy_months(n:int):
    ans=1
    if n ==12:
        ans= 0.8
    if n in [1,2,6,7]:
        ans=1.2
    return ans    
    
    

In [5]:
def number_of_tests(fecha): ### This takes a date and returns how many tests occur
    ans=int (busy_days(fecha.weekday())*busy_months(fecha.month)*np.random.uniform(0.95,1.05))
    return ans


## Generate a new row given a data

In [32]:
def make_row(fecha):############ This takesa a date and returns an entry in the Records_df
    type_of_test=random.choices(type_list,weights=[3,4])[0]
    adequacy=random.choices(adequacy_list,weights=[5,2,1])[0]
    result=random.choices(results_list,weights=[4,1,1,1,1])[0]
    gen_type=random.choice(genotype_list)
    cytology=random.choices(value_sign,weights=[4,1])[0]+"cytology"
    hystology="Not prescribed"
    if cytology=="Positivecytology":
        hystology=random.choices(value_sign,weights=[92,8])[0]+"hystology"
    need_surgery=0
    if cytology=="Negativecytology":
        need_surgery=random.choices([0,1],weights=[87,13])[0]
    
    new_row = pd.DataFrame([{'day':fecha.date(), 'type':type_of_test,'adequacy':adequacy, 'result':result,'genotype':gen_type,'cytology':cytology,'hystology':hystology,"need_surgery":need_surgery}])
    return new_row

## Choose the date range 

In [33]:
starting_date = date.today() - timedelta(days = 366)
ending_date=date.today() +timedelta(days = 20)

## Generate Records_df

In [34]:
def generate_test_df():
    columns=['day', 'type', 'adequacy', 'result', 'genotype', 'cytology', 'hystology']
    records_df = pd.DataFrame(columns=columns)
    daterange = pd.date_range(starting_date, ending_date, freq='D')
    for item in daterange:
        N_of_test=number_of_tests(item)
        for patient in range(N_of_test):
            new_row=make_row(item)
            records_df = pd.concat([records_df, new_row], axis=0, ignore_index=True)
    return records_df

In [35]:
Records_df=generate_test_df()

In [36]:
Records_df.shape

(34938, 8)

In [37]:
Records_df[Records_df["cytology"]=="Positivecytology"].shape

(28081, 8)

In [41]:
Records_df["need_surgery"].sum()

925.0

In [24]:
Records_df[Records_df["need_surgery"]==0].shape

(27975, 8)

### This is the dataframe that the app takes

In [42]:
Records_df.to_csv("NewRecords.csv")

### GENERATE FREQUENCY DATA FRAME

In [18]:
def new_row_frequency(df,fecha):
    columnas= list(Records_df.columns)[1:]
    new_row_in={"day":fecha}
    df_fecha=df[df["day"]==fecha]
    new_row_in["All"]=df_fecha.shape[0]
    for columna in columnas:
        column_values=list(df_fecha[columna].unique())
        for tipo in column_values:
            temp_df=df_fecha[df_fecha[columna]==tipo]
            new_row_in[tipo]=temp_df.shape[0]
    types=list(df_fecha["type"].unique())
    adequacies=list(df_fecha["adequacy"].unique())
    cytologies=list(df_fecha["cytology"].unique())
    hystologies=list(df_fecha["hystology"].unique())
    for tipo in types:
        for ad in adequacies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["adequacy"]==ad)]
            new_row_in[tipo+ad]=temp_df.shape[0]
        for cyt in cytologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["cytology"]==cyt)]
            new_row_in[tipo+cyt]=temp_df.shape[0]
        for hyst in hystologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["hystology"]==hyst)]
            new_row_in[tipo+hyst]=temp_df.shape[0]


        # new_row_in["weekday"]=week_day(fecha.weekday())
    
    return pd.DataFrame([new_row_in])

In [19]:
def make_frequency_df(df):
    fechas=df["day"].unique()
    freq_df=pd.DataFrame()
    for fecha in fechas:
        freq_df=pd.concat([freq_df,new_row_frequency(df,fecha)],axis=0, ignore_index=True)
    
    return freq_df

In [20]:
Frequency_df=make_frequency_df(Records_df)

In [17]:
#Frequency_df.to_csv('Frequency.csv')#